<h1 align=center><font size = 5>Neighborhoods comparison in Madrid and Barcelona</font></h1>

## Introduction/Business Problem

We will compare the two biggest cities in Spain: Madrid and Barcelona.
The Business problem we want to address is to discover the neighborhoods that are similar between the two cities related to the category and number of venues that they have.
We will cluster the neighborhoods using k-means and then will examine the clusters to find similarities between the two cities.
We will analyse what makes them more attractive to tourism.
The target audience are franchise operators that focus on tourism in these two cities and need to know where to open the franchise.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Neighborhoods of Madrid Names

In [2]:
url = 'https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid'
html = requests.get(url).content
df_list = pd.read_html(html)
df_mad = df_list[-1]
df_mad.head()

,Distrito,Número,Nombre,Superficie (km²)[2]​,Imagen
0,Centro,11,Palacio,"1,471 km²",NaN
1,Centro,12,Embajadores,"1,032 km²",NaN
2,Centro,13,Cortes,"0,592 km²",NaN
3,Centro,14,Justicia,"0,742 km²",NaN
4,Centro,15,Universidad,"0,947 km²",NaN


In [3]:
df_mad2= df_mad['Nombre']

df_madrid = df_mad2.to_frame()
df_madrid = df_madrid.rename(columns={'Nombre': 'Neighborhood'})
df_madrid.head()

,Neighborhood
0,Palacio
1,Embajadores
2,Cortes
3,Justicia
4,Universidad


In [4]:
df_madrid['Neighborhood'].isnull().values.any()

False

### Neighborhoods of Barcelona Names

In [5]:
url = 'https://es.wikipedia.org/wiki/Distritos_de_Barcelona'
html = requests.get(url).content
df_list = pd.read_html(html)
df_bcn = df_list[-2]
df_bcn = df_bcn.rename(columns={'Barrios (nº)': 'Neighborhood'})
df_bcn2= df_bcn['Neighborhood']
df_bcn = df_bcn2.to_frame()
df_bcn.head()

,Neighborhood
0,"El Raval (1), Barrio Gótico (2), La Barcelonet..."
1,"El Fort Pienc (5), Sagrada Familia (6), Dreta ..."
2,"Pueblo Seco (11), La Marina del Prat Vermell (..."
3,"Les Corts (19), La Maternidad y San Ramón (20)..."
4,"Vallvidrera, el Tibidabo i les Planes (22), Sa..."


In [6]:
#Export to csv to clean and prepare the data

df1=df_bcn2[0]+ ', '
for i in range(df_bcn2.shape[0]-1) :
      df1 = df1 + df_bcn2[i+1] + ', '    

with open("BcnBarrios.csv","w") as file:
    file.write(df1 + "\n")


In [7]:
df_bcn2 = pd.read_csv('BcnBarrios1.csv', encoding="latin-1")
df_barcelona = df_bcn2.T.reset_index()
df_barcelona.columns = ['Neighborhood']
df_barcelona.head()

,Neighborhood
0,El Raval
1,Barrio Gótico
2,La Barceloneta
3,Santa Caterina i la Ribera
4,El Fort Pienc


In [8]:
df_barcelona['Neighborhood'].isnull().values.any()

False

#### Shape of the Dataframes

In [9]:
df_madrid.shape

(131, 1)

In [10]:
df_barcelona.shape

(77, 1)

#### Use geopy library to get the latitude and longitude values of Madrid Neighborhoods.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below. </br> We will print the neighborhoods that give an error when getting the coordinates and we will not consider them in our study.

In [11]:
data = []
df_madll = pd.DataFrame(data, columns=['Neighborhood', 'Latitude', 'Longitude'])

for i in range(len(df_madrid)) :
    address = df_madrid.iloc[i,0]+', Madrid, Spain'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    
        df_madll = df_madll.append({'Neighborhood':df_madrid.iloc[i,0],'Latitude':latitude,'Longitude':longitude}, ignore_index=True)
   
    except: 
        print( df_madrid.iloc[i,0])

Los Cármenes
Valderrivas
Casco Histórico de Barajas


In [12]:
df_madll.head()

,Neighborhood,Latitude,Longitude
0,Palacio,40.415129,-3.715618
1,Embajadores,40.409681,-3.701644
2,Cortes,40.414779,-3.697584
3,Justicia,40.423957,-3.695747
4,Universidad,40.425409,-3.705989


#### Use geopy library to get the latitude and longitude values of Barcelona Neighborhoods.

In [13]:
data = []
df_bcnll = pd.DataFrame(data, columns=['Neighborhood', 'Latitude', 'Longitude'])

for i in range(len(df_barcelona)) :
    address = df_barcelona.iloc[i,0]+', Barcelona, Spain'
    try:
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    
        df_bcnll = df_bcnll.append({'Neighborhood':df_barcelona.iloc[i,0],'Latitude':latitude,'Longitude':longitude}, ignore_index=True)
    
    except: 
        print( df_barcelona.iloc[i,0])

La Antigua Izquierda del Ensanche
La Nueva Izquierda del Ensanche
Pueblo Seco
Vallcarca y los Penitentes
Villa de Gracia
El Valle de Hebrón
El Campo del Arpa del Clot
Pueblo Nuevo
San Martín de Provensals


In [14]:
df_bcnll.head()

,Neighborhood,Latitude,Longitude
0,El Raval,41.379518,2.168368
1,Barrio Gótico,41.383395,2.176912
2,La Barceloneta,41.380653,2.189927
3,Santa Caterina i la Ribera,41.386650,2.184194
4,El Fort Pienc,41.395925,2.182325


#### Create a map of Madrid with neighborhoods superimposed on top.

In [15]:
address = 'Madrid, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Madrid using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df_madll['Latitude'], df_madll['Longitude'], df_madll['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

#### Create a map of Barcelona with neighborhoods superimposed on top.

In [16]:
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df_bcnll['Latitude'], df_bcnll['Longitude'], df_bcnll['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

#### Define Foursquare Credentials and Version

We start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [17]:
CLIENT_ID = 'INTDEIQ04DX555YN1MTDX44V5CAPBR5ABIA5PBTZO0FDOZJI' # your Foursquare ID
CLIENT_SECRET = 'WFA4IQIH4SU5ZI3TLVVRBEEO1UUQ3YBHBPYNRPTVUN1RMRW1' # your Foursquare Secret
VERSION = '20180604'
##LIMIT = 30



## Explore Neighborhoods of Madrid

#### Let's explore the first Madrid neighborhood's venues

In [18]:
neighborhood_latitude = df_madll.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_madll.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_madll.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Palacio are 40.41512925, -3.7156179983990922.


#### Let's get the top 50 venues that are in this neighborhood within a radius of 200 meters.

In [19]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 200 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=INTDEIQ04DX555YN1MTDX44V5CAPBR5ABIA5PBTZO0FDOZJI&client_secret=WFA4IQIH4SU5ZI3TLVVRBEEO1UUQ3YBHBPYNRPTVUN1RMRW1&v=20180604&ll=40.41512925,-3.7156179983990922&radius=200&limit=50'

In [20]:
results = requests.get(url).json()
#results

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean json and structure it into a *pandas* dataframe.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Castro\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Santa Iglesia Catedral de Santa María la Real ...,Church,40.415767,-3.714516
1,Muralla musulmana de Madrid,Historic Site,40.414881,-3.714990
2,Tienda La Rebelión de los Mandiles,Wine Shop,40.415133,-3.713358
3,Terraza Atenas,Beer Garden,40.414559,-3.717352
4,Dehesa Santa María,Tapas Restaurant,40.415220,-3.713438


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods 

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Code to run the above function on each neighborhood and create a new dataframe called *dmadrid_venues*.

In [25]:
dmadrid_venues = getNearbyVenues(names=df_madll['Neighborhood'],
                                   latitudes=df_madll['Latitude'],
                                   longitudes=df_madll['Longitude']
                                  )

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepción
San Pascual

In [26]:
print('There are {} uniques categories.'.format(len(dmadrid_venues['Venue Category'].unique())))

There are 210 uniques categories.


In [27]:
# save data to csv
dmadrid_venues.to_csv("madrid_venues.csv")

#### Analyze Each Neighborhood of Madrid

In [28]:
# one hot encoding
dmadrid_onehot = pd.get_dummies(dmadrid_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dmadrid_onehot['Neighborhood'] = dmadrid_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dmadrid_onehot.columns[-1]] + list(dmadrid_onehot.columns[:-1])
dmadrid_onehot = dmadrid_onehot[fixed_columns]

dmadrid_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lottery Retailer,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trade School,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Palacio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Palacio,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group by neighborhood and normalize

In [29]:
dmadrid_grouped = dmadrid_onehot.groupby('Neighborhood').mean().reset_index()
dmadrid_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comedy Club,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Embassy / Consulate,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lottery Retailer,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Restaurant,Road,Russian Restaurant,Salad Place,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trade School,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store
0,Abrantes,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.00000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00000

##### Let's put it into a *pandas* dataframe

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dmadrid_grouped['Neighborhood']

for ind in np.arange(dmadrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dmadrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abrantes,Metro Station,Pizza Place,Grocery Store,Bakery,Fast Food Restaurant
1,Acacias,Spanish Restaurant,Café,Pizza Place,Restaurant,Supermarket
2,Adelfas,Spanish Restaurant,Breakfast Spot,Bakery,Diner,Coffee Shop
3,Alameda de Osuna,Tapas Restaurant,Italian Restaurant,Metro Station,Pizza Place,Smoke Shop
4,Almagro,Spanish Restaurant,Restaurant,Pub,French Restaurant,Salad Place


###  Cluster Madrid Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [32]:
# set number of clusters
kclusters = 6

dmadrid_grouped_clustering = dmadrid_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dmadrid_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 0, 3, 3, 3, 1])

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dmadrid_merged = df_madll

# merge madrid_grouped with madrid_data to add latitude/longitude for each neighborhood
dmadrid_merged = dmadrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dmadrid_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Palacio,40.415129,-3.715618,3.0,Plaza,Historic Site,Tapas Restaurant,Music Venue,Gastropub
1,Embajadores,40.409681,-3.701644,3.0,Bar,Tapas Restaurant,Coffee Shop,Theater,Restaurant
2,Cortes,40.414779,-3.697584,1.0,Spanish Restaurant,Hotel,Restaurant,Bar,Theater
3,Justicia,40.423957,-3.695747,3.0,Restaurant,Bakery,Spanish Restaurant,Hotel,Nightclub
4,Universidad,40.425409,-3.705989,3.0,Tapas Restaurant,Bar,Bookstore,Cocktail Bar,Coffee Shop


In [34]:
dmadrid_merged.to_excel("MadridMerged5.xlsx")

In [35]:
# Clean the rows that have nulls
df1 = dmadrid_merged.dropna(how='any',axis=0) 

Finally, let's visualize the resulting clusters

In [36]:
address = 'Madrid, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood'], df1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Madrid Clusters

#### Cluster 1

In [37]:
df1.loc[df1['Cluster Labels'] == 0, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,Almenara,Bar,Metro Station,Playground,Library,Food & Drink Shop
36,Berruguete,Tapas Restaurant,Bar,Deli / Bodega,Gym / Fitness Center,Supermarket
63,Las Águilas,Bar,Spanish Restaurant,Train Station,Shopping Mall,Seafood Restaurant
67,Vista Alegre,Breakfast Spot,Fast Food Restaurant,Pizza Place,Thrift / Vintage Store,Tapas Restaurant
68,Puerta Bonita,Pharmacy,Bar,Tapas Restaurant,Women's Store,Fried Chicken Joint
73,San Fermín,Spanish Restaurant,Bed & Breakfast,Shopping Mall,Tapas Restaurant,Bar
82,Portazgo,Bar,Gym / Fitness Center,BBQ Joint,Fast Food Restaurant,Frozen Yogurt Shop
98,Costillares,Gym / Fitness Center,Italian Restaurant,Pharmacy,Sports Bar,Bar
105,Villaverde Alto,Pizza Place,Bar,Women's Store,Frozen Yogurt Shop,Fried Chicken Joint
109,Los Ángeles,Bar,Women's Store,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint


#### Cluster 2

In [38]:
df1.loc[df1['Cluster Labels'] == 1, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Cortes,Spanish Restaurant,Hotel,Restaurant,Bar,Theater
16,Ibiza,Spanish Restaurant,Restaurant,Italian Restaurant,Tapas Restaurant,Seafood Restaurant
19,Recoletos,Spanish Restaurant,Restaurant,Italian Restaurant,Accessories Store,Furniture / Home Store
23,Lista,Spanish Restaurant,Restaurant,Seafood Restaurant,Mediterranean Restaurant,Burger Joint
24,Castellana,Spanish Restaurant,Restaurant,Tapas Restaurant,Coffee Shop,Hotel
29,Nueva España,Spanish Restaurant,Gym,Fast Food Restaurant,Big Box Store,Deli / Bodega
31,Bellas Vistas,Spanish Restaurant,Seafood Restaurant,Gourmet Shop,Gym,Bar
35,Valdeacederas,Dance Studio,Hobby Shop,Breakfast Spot,Spanish Restaurant,Disc Golf
38,Arapiles,Bar,Spanish Restaurant,Theater,Café,Restaurant
40,Almagro,Spanish Restaurant,Restaurant,Pub,French Restaurant,Salad Place


#### Cluster 3

In [39]:
df1.loc[df1['Cluster Labels'] == 2, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
51,Casa de Campo,BBQ Joint,Women's Store,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint
69,Buenavista,BBQ Joint,Women's Store,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint


#### Cluster 4

In [40]:
df1.loc[df1['Cluster Labels'] == 3, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Palacio,Plaza,Historic Site,Tapas Restaurant,Music Venue,Gastropub
1,Embajadores,Bar,Tapas Restaurant,Coffee Shop,Theater,Restaurant
3,Justicia,Restaurant,Bakery,Spanish Restaurant,Hotel,Nightclub
4,Universidad,Tapas Restaurant,Bar,Bookstore,Cocktail Bar,Coffee Shop
5,Sol,Hotel,Tapas Restaurant,Plaza,Frozen Yogurt Shop,Electronics Store
6,Imperial,Gym,Hotel,Tapas Restaurant,Gym / Fitness Center,Chinese Restaurant
7,Acacias,Spanish Restaurant,Café,Pizza Place,Restaurant,Supermarket
8,Chopera,Park,Grocery Store,Tapas Restaurant,Italian Restaurant,Bar
9,Legazpi,Bar,Argentinian Restaurant,Café,Plaza,Beer Garden
10,Delicias,Breakfast Spot,Restaurant,Flea Market,Science Museum,Women's Store


#### Cluster 5

In [41]:
df1.loc[df1['Cluster Labels'] == 4, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Fuentelarreina,Soccer Field,Women's Store,Fruit & Vegetable Store,Fried Chicken Joint,French Restaurant
119,Arcos,Bakery,Soccer Field,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint


#### Cluster 6

In [65]:
df1.loc[df1['Cluster Labels'] == 5, df1.columns[[0] + list(range(4, df1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
77,Pradolongo,Burger Joint,Women's Store,Department Store,Frozen Yogurt Shop,Fried Chicken Joint


In [42]:
df1.to_excel("MadridCluster6.xlsx")

## Explore Neighborhoods of Barcelona

#### Let's explore the first Barcelona neighborhood's venues

In [43]:
neighborhood_latitude = df_bcnll.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_bcnll.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_bcnll.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of El Raval are 41.3795176, 2.1683678.


#### Let's get the top 50 venues that are in this neighborhood within a radius of 200 meters.

In [44]:
LIMIT = 50 # limit of number of venues returned by Foursquare API

radius = 200 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=INTDEIQ04DX555YN1MTDX44V5CAPBR5ABIA5PBTZO0FDOZJI&client_secret=WFA4IQIH4SU5ZI3TLVVRBEEO1UUQ3YBHBPYNRPTVUN1RMRW1&v=20180604&ll=41.3795176,2.1683678&radius=200&limit=50'

In [45]:
results = requests.get(url).json()

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


Clean json and structure it into a *pandas* dataframe.

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Castro\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,La Robadora,Gastropub,41.379500,2.170463
1,Robadors 23,Jazz Club,41.379581,2.170603
2,Cera 23,Spanish Restaurant,41.378947,2.166180
3,A Tu Bola,Tapas Restaurant,41.380096,2.169054
4,33/45,Cocktail Bar,41.381059,2.167399


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

31 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods

In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Code to run the function *getNearbyVenues* on each neighborhood and create a new dataframe called *dbarcelona_venues*.

In [50]:
dbarcelona_venues = getNearbyVenues(names=df_bcnll['Neighborhood'],
                                   latitudes=df_bcnll['Latitude'],
                                   longitudes=df_bcnll['Longitude']
                                  )


print('There are {} uniques categories.'.format(len(dmadrid_venues['Venue Category'].unique())))

El Raval
Barrio Gótico
La Barceloneta
Santa Caterina i la Ribera
El Fort Pienc
Sagrada Familia
Dreta de l''Eixample
Sant Antoni
La Marina del Prat Vermell
La Marina de Port
La Font de la Guatlla
Hostafrancs
La Bordeta
Sants-Badal
Sants
Zona Franca
Montjuic
Les Corts
La Maternitat i Sant Ramon
Pedralbes
Vallvidrera
el Tibidabo
les Planes
Sarrià
Las Tres Torres
Sant Gervasi-Bonanova
Sant Gervasi-Galvany
El Putget i Farró
El Coll
La Salud
Camp d'en Grassot i Gràcia Nova
Baix Guinardó
Can Baró
El Guinardó
La Font d'en Fargues
El Carmel
La Teixonera
Sant Genís dels Agudells
Montbau
La Clota
Horta
Vilapicina y La Torre Llobeta
Porta
Turó de la Peira
Can Peguera
La Guineueta
Canyelles
Les Roquetes
Verdún
La Prosperitat
La Trinitat Nova
Torre Baró
Ciudad Meridiana
Vallbona
La Trinitat Vella
Baró de Viver
El Buen Pastor
San Andrés
La Sagrera
Congrés i els Indians
Navas
El Clot
El Parc i la Llacuna del Poblenou
La Vila Olímpica del Poblenou
Diagonal Mar i Front Marítim del Poblenou
El Besós y el

#### Analyze Each Neighborhood

In [51]:
# one hot encoding
dbarcelona_onehot = pd.get_dummies(dbarcelona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dbarcelona_onehot['Neighborhood'] = dbarcelona_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dbarcelona_onehot.columns[-1]] + list(dbarcelona_onehot.columns[:-1])
dbarcelona_onehot = dbarcelona_onehot[fixed_columns]

dbarcelona_onehot.head()


,Yoga Studio,Accessories Store,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridge,Burger Joint,Bus Stop,Cafeteria,Café,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Neighborhood,Nightclub,Organic Grocery,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Peruvian Restaurant,Pet Store,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shop & Service,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Park Ride / Attraction,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,El Raval,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Group by neighborhood

In [52]:
dbarcelona_grouped = dbarcelona_onehot.groupby('Neighborhood').mean().reset_index()
dbarcelona_grouped

,Neighborhood,Yoga Studio,Accessories Store,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridge,Burger Joint,Bus Stop,Cafeteria,Café,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Organic Grocery,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Peruvian Restaurant,Pet Store,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Public Art,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Seafood Restaurant,Shop & Service,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Theme Park Ride / Attraction,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Wine Shop
0,Baix Guinardó,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.000,0.0,0.166667,0.00,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000
1,Barrio Gótico,0.000000,0.000000,0.000000,0.000000,0.000000,0.03125,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.03125,0.000000,0.000000,0.000000,0.031250,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.031250,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

##### Let's put it into a *pandas* dataframe

In [53]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dbarcelona_grouped['Neighborhood']

for ind in np.arange(dbarcelona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dbarcelona_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Baix Guinardó,Plaza,Beer Garden,Grocery Store,Park,Food & Drink Shop
1,Barrio Gótico,Spanish Restaurant,Plaza,Wine Bar,Hotel,Tea Room
2,Camp d'en Grassot i Gràcia Nova,Supermarket,Fruit & Vegetable Store,Basketball Court,Bar,Bakery
3,Can Baró,Dessert Shop,Bar,Plaza,Italian Restaurant,Grocery Store
4,Canyelles,Restaurant,Bakery,Mediterranean Restaurant,Spanish Restaurant,Wine Shop


###  Cluster Barcelona Neighborhoods

In [54]:
# set number of clusters
kclusters = 6

dbarcelona_grouped_clustering = dbarcelona_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dbarcelona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 0, 5, 0, 0, 1, 0, 0])

In [55]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dbarcelona_merged = df_bcnll

# merge barcelona_grouped with barcelona_data to add latitude/longitude for each neighborhood
dbarcelona_merged = dbarcelona_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dbarcelona_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,El Raval,41.379518,2.168368,0.0,Tapas Restaurant,Cocktail Bar,Spanish Restaurant,Dance Studio,Hotel
1,Barrio Gótico,41.383395,2.176912,4.0,Spanish Restaurant,Plaza,Wine Bar,Hotel,Tea Room
2,La Barceloneta,41.380653,2.189927,0.0,Tapas Restaurant,Spanish Restaurant,Mediterranean Restaurant,Bar,Plaza
3,Santa Caterina i la Ribera,41.386650,2.184194,0.0,Tapas Restaurant,Hotel,Mexican Restaurant,Italian Restaurant,Arepa Restaurant
4,El Fort Pienc,41.395925,2.182325,0.0,Gym / Fitness Center,Supermarket,Restaurant,Mexican Restaurant,Mediterranean Restaurant


In [56]:
dbarcelona_merged.to_excel("BarcelonaMerged6.xlsx")

In [57]:
df2 = dbarcelona_merged.dropna(how='any',axis=0)
df2.to_excel("BarcelonaCluster6.xlsx")

Finally, let's visualize the resulting clusters

In [58]:
address = 'Barcelona, Spain'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Barcelona Clusters

#### Cluster 1

In [59]:
df2.loc[df2['Cluster Labels'] == 0, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,El Raval,Tapas Restaurant,Cocktail Bar,Spanish Restaurant,Dance Studio,Hotel
2,La Barceloneta,Tapas Restaurant,Spanish Restaurant,Mediterranean Restaurant,Bar,Plaza
3,Santa Caterina i la Ribera,Tapas Restaurant,Hotel,Mexican Restaurant,Italian Restaurant,Arepa Restaurant
4,El Fort Pienc,Gym / Fitness Center,Supermarket,Restaurant,Mexican Restaurant,Mediterranean Restaurant
5,Sagrada Familia,Coffee Shop,Mexican Restaurant,Historic Site,Pizza Place,Plaza
6,Dreta de l''Eixample,Hotel,Bookstore,Beer Bar,Boutique,Gastropub
7,Sant Antoni,Spanish Restaurant,Mediterranean Restaurant,Café,Coffee Shop,Italian Restaurant
9,La Marina de Port,Bakery,Spanish Restaurant,Sporting Goods Shop,Grocery Store,Café
10,La Font de la Guatlla,Hotel,Spanish Restaurant,Hostel,Sports Club,Convenience Store
11,Hostafrancs,Café,Restaurant,Pizza Place,Burger Joint,Bar


#### Cluster 2

In [60]:
df2.loc[df2['Cluster Labels'] == 1, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
56,El Buen Pastor,Café,Metro Station,Mediterranean Restaurant,Wine Shop,Football Stadium
64,Diagonal Mar i Front Marítim del Poblenou,Mediterranean Restaurant,Restaurant,Hotel,Playground,Beer Garden
67,La Verneda y la Paz,Café,Tapas Restaurant,Wine Shop,Football Stadium,Food & Drink Shop


#### Cluster 3

In [61]:
df2.loc[df2['Cluster Labels'] == 2, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,Sant Genís dels Agudells,Supermarket,Wine Shop,Football Stadium,Food & Drink Shop,Food


#### Cluster 4

In [62]:
df2.loc[df2['Cluster Labels'] == 3, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
39,La Clota,Garden Center,Wine Shop,Falafel Restaurant,Food & Drink Shop,Food


#### Cluster 5

In [63]:
df2.loc[df2['Cluster Labels'] == 4, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Barrio Gótico,Spanish Restaurant,Plaza,Wine Bar,Hotel,Tea Room
8,La Marina del Prat Vermell,Gastropub,Restaurant,Mediterranean Restaurant,Plaza,Bus Stop
20,Vallvidrera,Photography Studio,Plaza,Ice Cream Shop,National Park,Light Rail Station
25,Sant Gervasi-Bonanova,Mediterranean Restaurant,College Gym,Sports Club,Wine Shop,Falafel Restaurant
31,Baix Guinardó,Plaza,Beer Garden,Grocery Store,Park,Food & Drink Shop
35,El Carmel,Plaza,Coffee Shop,Grocery Store,Supermarket,Wine Shop
36,La Teixonera,Tapas Restaurant,Bakery,Plaza,Bar,Food & Drink Shop
38,Montbau,Restaurant,Argentinian Restaurant,Plaza,Tennis Court,Basketball Court
42,Porta,Tapas Restaurant,Plaza,Public Art,Wine Shop,Flower Shop
47,Les Roquetes,Plaza,Gym Pool,Gym,Metro Station,Castle


#### Cluster 6

In [64]:
df2.loc[df2['Cluster Labels'] == 5, df2.columns[[0] + list(range(4, df2.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Les Corts,Restaurant,Bar,Hotel,Burger Joint,Plaza
29,La Salud,Restaurant,Wine Shop,Diner,Food,Flower Shop
46,Canyelles,Restaurant,Bakery,Mediterranean Restaurant,Spanish Restaurant,Wine Shop
48,Verdún,Restaurant,Argentinian Restaurant,Grocery Store,Diner,Food
66,Provençals del Poblenou,Auto Workshop,Spanish Restaurant,Liquor Store,Restaurant,Electronics Store
